## 以下脚本用于部署 Allin1 WebUI API SNS Notify

In [1]:
import boto3,os
from botocore.exceptions import ClientError

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name

dlq_name = f'allinone-api-dlq'
topic_name = f'allinone-api-notification'
# sns_https_endpoint = f"https://<YOUR_URL>/<PATH>"

print(account_id)
print(region_name)
print(dlq_name)
print(topic_name)

871839178153
ap-northeast-1
allinone-api-queue
allinone-api-notification


### 新建/获取 SNS topic

In [2]:
# snippet-start:[python.example_code.sns.SnsWrapper]
class SnsWrapper:
    """Encapsulates Amazon SNS topic and subscription functions."""
    def __init__(self, sns_resource):
        """
        :param sns_resource: A Boto3 Amazon SNS resource.
        """
        self.sns_resource = sns_resource
# snippet-end:[python.example_code.sns.SnsWrapper]

# snippet-start:[python.example_code.sns.CreateTopic]
    def create_topic(self, name):
        """
        Creates a notification topic.

        :param name: The name of the topic to create.
        :return: The newly created topic.
        """
        try:
            topic = self.sns_resource.create_topic(Name=name)
            print("Created topic with ARN:", name, topic.arn)
        except ClientError:
            print("Couldn't create topic:", name)
            raise
        else:
            return topic
# snippet-end:[python.example_code.sns.CreateTopic]

    @staticmethod
# snippet-start:[python.example_code.sns.Subscribe]
    def subscribe(topic, protocol, endpoint):
        """
        Subscribes an endpoint to the topic. Some endpoint types, such as email,
        must be confirmed before their subscriptions are active. When a subscription
        is not confirmed, its Amazon Resource Number (ARN) is set to
        'PendingConfirmation'.

        :param topic: The topic to subscribe to.
        :param protocol: The protocol of the endpoint, such as 'sms' or 'email'.
        :param endpoint: The endpoint that receives messages, such as a phone number
                         (in E.164 format) for SMS messages, or an email address for
                         email messages.
        :return: The newly added subscription.
        """
        
        delivery_policy = '''{
                      "healthyRetryPolicy": {
                        "minDelayTarget": 20,
                        "maxDelayTarget": 20,
                        "numRetries": 6,
                        "numMaxDelayRetries": 0,
                        "numNoDelayRetries": 0,
                        "numMinDelayRetries": 0,
                        "backoffFunction": "linear"
                      },
                      "sicklyRetryPolicy": null,
                      "throttlePolicy": null,
                      "requestPolicy": {
                        "headerContentType": "application/json; charset=utf-8"
                      },
                      "guaranteed": false
                    }
                    '''
        attributes={
            "RawMessageDelivery": "true",
            "DeliveryPolicy": delivery_policy
        }
        
        try:
            subscription = topic.subscribe(
                Protocol=protocol, Endpoint=endpoint, Attributes=attributes, ReturnSubscriptionArn=True)
            print(f"Subscribed {protocol} {endpoint} to topic {topic.arn}.")
        except ClientError:
            print(f"Couldn't subscribe {protocol} {endpoint} to topic {topic.arn}.")
            raise
        else:
            return subscription
# snippet-end:[python.example_code.sns.Subscribe]

sns_wrapper = SnsWrapper(boto3.resource('sns'))
print(f"Creating topic {topic_name}.")
topic = sns_wrapper.create_topic(topic_name)

Creating topic allinone-api-notification.
Created topic with ARN: allinone-api-notification arn:aws:sns:ap-northeast-1:871839178153:allinone-api-notification


### 为SNS Topic 添加订阅，支持SQS/Http/Https等方式

In [4]:
print(f"Subscribing https endpoint to {topic_name}.")
# http_sub = sns_wrapper.subscribe(topic, 'https', sns_https_endpoint)

Subscribing http endpoint to allinone-api-notification.
Subscribed https https://openapi.uqualities.com/AIGCImgCallBackServ/aigc/openApi/callback/info to topic arn:aws:sns:ap-northeast-1:871839178153:allinone-api-notification.


### （可选）新建 SQS

In [2]:
# snippet-start: [SQS]
sqs_resource = boto3.resource('sqs')

# snippet-start:[python.example_code.sqs.CreateQueue]
def create_queue(name, attributes=None):
    """
    Creates an Amazon SQS queue.

    :param name: The name of the queue. This is part of the URL assigned to the queue.
    :param attributes: The attributes of the queue, such as maximum message size or
                       whether it's a FIFO queue.
    :return: A Queue object that contains metadata about the queue and that can be used
             to perform queue operations like sending and receiving messages.
    """
    if not attributes:
        attributes = {}

    try:
        queue = sqs_resource.create_queue(
            QueueName=name,
            Attributes=attributes
        )
        print("Created queue with URL:", name, queue.url)
    except ClientError as error:
        print("Couldn't create queue named:", name)
        raise error
    else:
        return queue
    
# snippet-end:[python.example_code.sqs.CreateQueue]
queue = create_queue(
    dlq_name
)
print(f"Created queue with URL: {queue.url}.")
    
# snippet-end: [SQS]

Created queue with URL: allinone-api-queue https://sqs.ap-northeast-1.amazonaws.com/871839178153/allinone-api-queue
Created queue with URL: https://sqs.ap-northeast-1.amazonaws.com/871839178153/allinone-api-queue.


### （可选）为SQS添加Policy, 允许来自Topic的SendMessage调用

In [4]:
sqs_policy = '''{
  "Version": "2012-10-17",
  "Id": "__default_policy_ID",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sns.amazonaws.com"
      },
      "Action": "sqs:SendMessage",
      "Resource": "%s",
      "Condition": {
        "ArnEquals": {
          "aws:SourceArn": "%s"
        }
      }
    }
  ]
}''' % (queue.attributes['QueueArn'], topic.arn)

attr = {
        "Policy": sqs_policy
    }

print(f"Adding SQS policy to {dlq_name}.")
queue.set_attributes(
    Attributes=attr
)

# ---------------------------------------

Subscribing allinone-api-queue to allinone-api-notification.
Subscribed sqs arn:aws:sqs:ap-northeast-1:871839178153:allinone-api-queue to topic arn:aws:sns:ap-northeast-1:871839178153:allinone-api-notification.
Adding SQS policy to allinone-api-queue.
Done.


### （可选）新增SQS订阅

In [ ]:
print(f"Subscribing {dlq_name} to {topic_name}.")
sqs_sub = sns_wrapper.subscribe(topic, 'sqs', queue.attributes['QueueArn'])

print("Done.")